## Project Overview


This project aims to fine-tune the AraGPT2-large model on classical Islamic texts, specifically focusing on the book Al-Tadmuriyah by Shaykh al-Islam Ibn Taymiyyah. The goal is to have the model become highly specialized in predicting and generating text within this specific book, even if it leads to overfitting. Overfitting is acceptable and even encouraged for this task, as the primary objective is for the model to accurately represent and generate the unique language style and content of *Al-Tadmuriyah*.


To achieve this, I have followed these steps:

1. Data Collection: The text of Al-Tadmuriyah was scraped and processed for training. The code used for this scraping can be found on my [GitHub repository](#), and the processed dataset is available on Hugging Face datasets for public access.
   
2. Model Selection: I used the AraGPT2-large model from Hugging Face, a powerful language model for Arabic. The model was quantized to 4-bit precision to reduce computational load while still retaining its performance potential.

3. Training and Fine-Tuning: I trained the model for 50 epochs, initially observing poor performance. To address this, I performed a grid search over several hyperparameters to identify the best configuration for this task.

4. Model Evaluation and Logs: The results from different configurations are logged and visualized using TensorBoard. All models generated from the grid search are available on my [Hugging Face repository](#).

### What to Expect

In this notebook, you will find:
- Details on the data preparation process, including how the dataset was scraped and preprocessed.
- The fine-tuning steps applied to AraGPT2-large on Al-Tadmuriyah.
- A discussion of the initial training results, followed by the grid search for optimal hyperparameters.
- Insights into the performance of the model across different configurations, along with links to the trained models.
- Logs and visualizations of the training process using TensorBoard.


In [ ]:
! pip install -U transformers accelerate BitsAndBytes datasets
! pip install huggingface-hub arabert peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset
import torch
from arabert.preprocess import ArabertPreprocessor

In [ ]:
# Signing in so we can upload the model to the huggingface hub ..
from huggingface_hub import notebook_login

notebook_login()

### Explanation of the Code

1. **`BitsAndBytesConfig`**:
   This configuration is used to enable **4-bit quantization** to make the model more memory-efficient and speed up training. Key parameters include:
   - **`load_in_4bit=True`**: Loads the model with 4-bit precision to reduce memory usage.
   - **`bnb_4bit_use_double_quant=True`**: Enables double quantization for better performance and compression.
   - **`bnb_4bit_quant_type='nf4'`**: Specifies the quantization type as "NF4" (a type of quantization that provides better precision than traditional quantization methods).
   - **`bnb_4bit_compute_dtype=torch.float16`**: Specifies the data type used for computations, which is set to 16-bit floating point (float16) to further reduce memory usage and speed up computations.

2. **`check_point = 'aubmindlab/aragpt2-large'`**:
   The checkpoint for the `AraGPT2-large` model from `aubmindlab`, which is a large-scale Arabic language model.

3. **`arabert_prep = ArabertPreprocessor(model_name=check_point)`**:
   This initializes an **`ArabertPreprocessor`** for Arabic text preprocessing using the `AraGPT2-large` model. It prepares the text by applying tokenization and other necessary preprocessing steps specific to the Arabic language.


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16)

check_point = 'aubmindlab/aragpt2-large'
arabert_prep = ArabertPreprocessor(model_name=check_point)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(check_point, quantization_config=bnb_config,
                                             trust_remote_code=True)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(check_point, trust_remote_code=True)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--aubmindlab--aragpt2-large/snapshots/b870598f32c15f993567a09c977cc0e5431d28f0/config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--aubmindlab--aragpt2-large/snapshots/b870598f32c15f993567a09c977cc0e5431d28f0/config.json
Model config AraGPT2Config {
  "_name_or_path": "aubmindlab/aragpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "AraGPT2LMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attn_pdrop": 0.1,
  "auto_map": {
    "AutoConfig": "aubmindlab/aragpt2-large--configuration_aragpt2.AraGPT2Config",
    "AutoModel": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2Model",
    "AutoModelForCausalLM": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2LMHeadModel"
  },
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "initializer_range": 0.0141

## Fixing the Tokenizer

### Issue:

The tokenizer does not have a `pad_token`, and the `pad_token_id` is set to the same value as the `eos_token_id`.

These two issues could cause strange behavior when fine-tuning the model. To resolve this, we need to manually set a unique `pad_token`.

### Steps Taken:

1. **Manually set the `pad_token`** to ensure that the tokenizer has a unique padding token.
2. **Set a unique `pad_token_id`** to avoid conflicts with the `eos_token_id`.

### Encountered Error:

After adjusting the tokenizer and attempting to load the model for evaluation, I encountered a **shape mismatch error**:

```
RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
size mismatch for base_model.model.transformer.wte.weight: copying a param with shape torch.Size([64001, 1280]) from checkpoint, the shape in current model is torch.Size([64000, 1280]).
size mismatch for base_model.model.lm_head.weight: copying a param with shape torch.Size([64001, 1280]) from checkpoint, the shape in current model is torch.Size([64000, 1280]).
```

### Solution:

To resolve this issue, I realized that I could simply delete the `pad_token` before uploading the model to the Hugging Face Hub. This ensures the tokenizer and model remain compatible without causing further errors.

---



In [ ]:
print(f'tokenizer len befour padding token: {len(tokenizer)}')
tokenizer.add_special_tokens({'pad_token':'[PAD]'}) # adding the padding token
print(f'len after adding the new token: {len(tokenizer)}')

tokenizer len befour padding token: 64000
len after adding the new token: 64001


In [ ]:
# changing the toekenizer len in the model config to make sure it has been modeified and got the padding token ..
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id # changin the pad token id in the model.config

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 64001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [ ]:
# Verfy the changes:
print(f'padding token ID in the tokenizer: {tokenizer.pad_token_id}')
print(f'padding token ID in the model config {model.config.pad_token_id}')
print(f'the padding token from the tokenizer: {tokenizer.pad_token}')
print(f'eos token ID in the model: {model.config.eos_token_id}, eos token ID in the tokenizer {tokenizer.eos_token_id}')
print(f'the tokenizer len:{len(tokenizer)}, the model input ebedding layer:{model.get_input_embeddings()}')
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token, tokenizer.eos_token

padding token ID in the tokenizer: 64000
padding token ID in the model config 64000
the padding token from the tokenizer: [PAD]
eos token ID in the model: 0, eos token ID in the tokenizer 0
the tokenizer len:64001, the model input ebedding layer:Embedding(64001, 1280)


### LoRA

In this code, we are preparing a model for k-bit training using Low-Rank Adaptation (LoRA), which helps reduce the number of trainable parameters. This technique is particularly useful for large models like GPT-2, allowing fine-tuning with fewer resources.

1. **`prepare_model_for_kbit_training(model)`**: Prepares the model for k-bit quantization, reducing its memory footprint and speeding up training.
   
2. **`LoraConfig`**: Defines the configuration for LoRA:
   - **`task_type=TaskType.CAUSAL_LM`**: Specifies that the task is causal language modeling, appropriate for GPT-2.
   - **`inference_mode=False`**: Indicates that the model will be trained, not just used for inference.
   - **`r=8`**: The rank of the LoRA matrices, controlling the extent of parameter reduction.
   - **`lora_alpha=32`**: A scaling factor that controls the impact of LoRA parameters.
   - **`lora_dropout=0.05`**: Dropout probability to prevent overfitting.
   - **`target_modules`**: Specifies which layers to apply LoRA to (attention and MLP layers).

3. **`get_peft_model(model, lora_config)`**: Wraps the original model with the LoRA configuration, enabling parameter-efficient fine-tuning.


In [ ]:
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Since GPT-2 is a causal language model
    inference_mode=False,          # Set to True if only doing inference
    r=8,                           # Rank of the LoRA matrices
    lora_alpha=32,                 # Scaling factor
    lora_dropout=0.05,              # Dropout probability
    target_modules=[
        "attn.c_attn",  # Self-attention projection (q, k, v)
        "attn.c_proj",  # Self-attention output projection
        "mlp.c_fc",     # MLP intermediate projection
        "mlp.c_proj"    # MLP output projection
    ]
)
model = get_peft_model(model, lora_config)

Loading the data then tokenize it

In [ ]:
def tokenizer_function(exampls):
  cleaned_text = [arabert_prep.preprocess(text) for text in exampls['combined']]
  return tokenizer(cleaned_text, padding=True, truncation=True, max_length=1024)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset('ahmadAlrabghi/al_tadmoreyyah')
print(raw_dataset)
train_dataset = raw_dataset.select_columns(['combined'])
train_dataset = train_dataset.map(tokenizer_function, batched=True)
print(train_dataset)

Using the latest cached version of the dataset since ahmadAlrabghi/al_tadmoreyyah couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/ahmadAlrabghi___al_tadmoreyyah/default/0.0.0/7a295c83c22d7ce96678af1b35bfea88baccde0c (last modified on Fri Sep  6 15:31:53 2024).


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'page', 'title', 'text', 'cleaned_text', 'len_cleand', 'combined', 'len_combined'],
        num_rows: 81
    })
})


Map:   0%|          | 0/81 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['combined', 'input_ids', 'attention_mask'],
        num_rows: 81
    })
})


In [ ]:
training_args = TrainingArguments(learning_rate=2e-6, per_device_train_batch_size=16,
                                  num_train_epochs=1, output_dir='ahmadAlrabghi/al_tadmoreyyah_model',
                                  warmup_steps=round(6 * 0.2), # about 20% of the steps for the first epoch
                                  optim='paged_adamw_8bit', # using the 8 bit optimizer for computational efficiency
                                  per_device_eval_batch_size=4,
                                  weight_decay=0.01, push_to_hub=False, # dont push the model until we remove the pad token !
                                  hub_model_id='ahmadAlrabghi/al_tadmoreyyah_model',
                                  report_to='all', log_level='info',
                                  evaluation_strategy='no', fp16=True,
                                  logging_strategy='epoch', save_strategy='epoch')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices


In [ ]:
trainer = Trainer(model=model, tokenizer=tokenizer, data_collator=data_collator,
                  args=training_args, train_dataset=train_dataset['train'])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Using auto half precision backend


In [ ]:
%%time

trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: combined. If combined are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 81
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6
  Number of trainable parameters = 5,898,240
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, 

Step,Training Loss
6,3.689000


Saving model checkpoint to ahmadAlrabghi/al_tadmoreyyah_model/checkpoint-6
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--aubmindlab--aragpt2-large/snapshots/b870598f32c15f993567a09c977cc0e5431d28f0/config.json
Model config AraGPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "AraGPT2LMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attn_pdrop": 0.1,
  "auto_map": {
    "AutoConfig": "aubmindlab/aragpt2-large--configuration_aragpt2.AraGPT2Config",
    "AutoModel": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2Model",
    "AutoModelForCausalLM": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2LMHeadModel"
  },
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "initializer_range": 0.014142135623731,
  "intermediate_size": 5120,
  "layer_norm_epsilon": 1e-05,
  "model_type": "aragpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,


CPU times: user 58.8 s, sys: 3.04 s, total: 1min 1s
Wall time: 1min 41s


TrainOutput(global_step=6, training_loss=3.6890131632486978, metrics={'train_runtime': 101.33, 'train_samples_per_second': 0.799, 'train_steps_per_second': 0.059, 'total_flos': 219395184353280.0, 'train_loss': 3.6890131632486978, 'epoch': 1.0})

## Removing the Padding Token

In this section of the code, we are removing the **`pad_token`** that was previously added to the tokenizer. This is necessary to avoid any size mismatch errors when loading the trained model due to the padding token being present in the tokenizer but not required during inference.

### Steps Explained:

1. **Get the current vocabulary**:
   - We first retrieve the tokenizer's current vocabulary using the `get_vocab()` method.

2. **Check if the `pad_token` exists**:
   - If the `pad_token` (`'[PAD]'`) exists in the vocabulary, we remove it using the `pop()` method.

3. **Rebuild the vocabulary**:
   - After removing the `pad_token`, we rebuild the vocabulary (`new_vocab`) by creating a list of the remaining tokens.

4. **Reinitialize the tokenizer without the `pad_token`**:
   - We then recreate the tokenizer using the updated vocabulary that no longer includes the `pad_token` by calling `AutoTokenizer.from_pretrained`.

5. **Update the model configuration**:
   - The model's configuration is updated by setting `pad_token_id` to `None`, ensuring that the model is no longer dependent on the padding token.
   - We also adjust the model’s token embeddings to match the new vocabulary size using `resize_token_embeddings`.



This process ensures that both the model and the tokenizer are in sync after removing the padding token, preventing any potential size mismatch errors when the model is used.

In [ ]:
vocab = tokenizer.get_vocab()

if '[PAD]' in vocab:
    vocab.pop('[PAD]')

new_vocab = list(vocab.keys())

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer.name_or_path,
    vocab=new_vocab, trust_remote_code=True
)

model.config.pad_token_id = None
model.resize_token_embeddings(len(tokenizer))
# verify the changes
# print(f'padding token in model: {model.config.pad_token_id}, padding token in the tokenizer: {tokenizer.pad_token}')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--aubmindlab--aragpt2-large/snapshots/b870598f32c15f993567a09c977cc0e5431d28f0/config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--aubmindlab--aragpt2-large/snapshots/b870598f32c15f993567a09c977cc0e5431d28f0/config.json
Model config AraGPT2Config {
  "_name_or_path": "aubmindlab/aragpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "AraGPT2LMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attn_pdrop": 0.1,
  "auto_map": {
    "AutoConfig": "aubmindlab/aragpt2-large--configuration_aragpt2.AraGPT2Config",
    "AutoModel": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2Model",
    "AutoModelForCausalLM": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2LMHeadModel"
  },
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_toke

Embedding(64000, 1280)

In [ ]:
# verify the changes
print(f'padding token ID in the tokenizer: {tokenizer.pad_token_id}')
print(f'padding token ID in the model config {model.config.pad_token_id}')
print(f'padding token in model: {model.config.pad_token_id}')
print(f'the padding token from the tokenizer: {tokenizer.pad_token}')
print(f'eos token ID in the model: {model.config.eos_token_id}, eos token ID in the tokenizer {tokenizer.eos_token_id}')
print(f'the tokenizer len:{len(tokenizer)}, the model input ebedding layer:{model.get_input_embeddings()}')


padding token ID in the tokenizer: None
padding token ID in the model config None
padding token in model: None
the padding token from the tokenizer: None
eos token ID in the model: 0, eos token ID in the tokenizer 0
the tokenizer len:64000, the model input ebedding layer:Embedding(64000, 1280)


In [ ]:
trainer.push_to_hub()

Saving model checkpoint to ahmadAlrabghi/al_tadmoreyyah_model
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--aubmindlab--aragpt2-large/snapshots/b870598f32c15f993567a09c977cc0e5431d28f0/config.json
Model config AraGPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "AraGPT2LMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attn_pdrop": 0.1,
  "auto_map": {
    "AutoConfig": "aubmindlab/aragpt2-large--configuration_aragpt2.AraGPT2Config",
    "AutoModel": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2Model",
    "AutoModelForCausalLM": "aubmindlab/aragpt2-large--modeling_aragpt2.AraGPT2LMHeadModel"
  },
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "initializer_range": 0.014142135623731,
  "intermediate_size": 5120,
  "layer_norm_epsilon": 1e-05,
  "model_type": "aragpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": 

events.out.tfevents.1725645141.2749de5fb9ec.20791.13:   0%|          | 0.00/7.68k [00:00<?, ?B/s]

events.out.tfevents.1725645650.2749de5fb9ec.20791.14:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/23.6M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1725646130.2749de5fb9ec.20791.15:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmadAlrabghi/al_tadmoreyyah_model/commit/bceab286f3a59202aad86272ff2664c3b72c2195', commit_message='End of training', commit_description='', oid='bceab286f3a59202aad86272ff2664c3b72c2195', pr_url=None, pr_revision=None, pr_num=None)

### Model Performance Evaluation

The performance of this baseline model is actually **very poor**.  
You can try running the model to observe its performance.

To improve the model, we need to perform a **grid search** to identify the best hyperparameters that will optimize the model's performance for our specific data.


# Grid Search Step:

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset
import torch
from arabert.preprocess import ArabertPreprocessor
import itertools
import os
from huggingface_hub import Repository

In [13]:
# remove unwanted files to free up some space
! rm -r sample_data

rm: cannot remove 'sample_data': No such file or directory
rm: cannot remove 'ahmadAlrabghi': No such file or directory


In [ ]:
# Mount Google Drive to save logging files during training
from google.colab import drive
drive.mount('/content/drive')
# or you could just download it very simply

from google.colab import files
# files.download(path)

Mounted at /content/drive


In [14]:
# debug cuda errors if needed:
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

In [15]:
%%time
# the only difference in the code here is in the gridsearch part and all of the other parts are the same.

# Hyperparameter grids, # I have use a small grid due to coputational rss limits:
learning_rates = [5e-5, 2e-4]
batch_sizes = [8, 4, 2]
weight_decays = [0.01, 0.0]
grad_accumulation_steps = [2]

# LoRA-specific hyperparameters
lora_ranks = [32, 64]
lora_alphas = [64, 128]
lora_dropouts = [0.0]



# Function to train the model with a given set of hyperparameters
def train_model(lr, batch_size, weight_decay, grad_accum_steps, lora_r, lora_alpha, lora_dropout, check_point, dataset_path):
    print('start trianing function')

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16
    )

    model = AutoModelForCausalLM.from_pretrained(check_point, trust_remote_code=True, quantization_config=bnb_config)
    tokenizer = AutoTokenizer.from_pretrained(check_point, trust_remote_code=True)
    arabert_prep = ArabertPreprocessor(model_name=check_point)



    # adding the padding token:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

    # peft:
    model = prepare_model_for_kbit_training(model)
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=[
            "attn.c_attn",
            "attn.c_proj",
            "mlp.c_fc",
            "mlp.c_proj"
        ]
    )
    model = get_peft_model(model, lora_config)



    # Data
    def tokenizer_function(exampls):
        cleaned_text = [arabert_prep.preprocess(text) for text in exampls['combined']]
        return tokenizer(cleaned_text, padding=True, truncation=True, max_length=1024)
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    from datasets import load_dataset
    raw_dataset = load_dataset(dataset_path)
    train_dataset = raw_dataset.select_columns(['combined'])
    train_dataset = train_dataset.map(tokenizer_function, batched=True)
    output_dir = f"./models/al_tadmoreyyah_lr{lr}_bs{batch_size}_wa{weight_decay}_ga{grad_accum_steps}_r{lora_r}_alpha{lora_alpha}_dropout{lora_dropout}"

    # saving the logging data to specific file to use it later in tensorboard:
    logging_dir = f"./logs/al_tadmoreyyah_lr{lr}_bs{batch_size}_wa{weight_decay}_ga{grad_accum_steps}_r{lora_r}_alpha{lora_alpha}_dropout{lora_dropout}"

    # getting the right number to use in in warmup hyperparameter in training arguments:
    datasize = 81
    warm_up  = 81 / batch_size



    training_args = TrainingArguments(
        output_dir='ahmadAlrabghi/al_tadmoreyyah_model',
        per_device_train_batch_size=batch_size,
        num_train_epochs=15,
        weight_decay=weight_decay,
        learning_rate=lr,
        gradient_accumulation_steps=grad_accum_steps,
        logging_dir=logging_dir,
        logging_steps=5,
        logging_strategy='steps',
        save_steps=10,
        warmup_steps=round(warm_up),
        optim='paged_adamw_8bit',
        push_to_hub=False,
        hub_model_id='ahmadAlrabghi/al_tadmoreyyah_model',
        report_to='tensorboard',
        # log_level='info',
        evaluation_strategy='no'
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset['train'],
        data_collator=data_collator,
        tokenizer=tokenizer
    )

    print('training_started...')
    trainer.train()

    # removing the pad token to save the model
    vocab = tokenizer.get_vocab()
    if '[PAD]' in vocab:
        vocab.pop('[PAD]')

    new_vocab = list(vocab.keys())

    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer.name_or_path,
        vocab=new_vocab, trust_remote_code=True
    )

    model.config.pad_token_id = None
    model.resize_token_embeddings(len(tokenizer))

    trainer.push_to_hub(
        model_name='ahmadAlrabghi/al_tadmoreyyah_model',
        # adding tags to differentiation between the models
        tags=
         [
             f'lr:{lr}', f'epochs:{20}', f'lora-dropout:{lora_dropout}', f'train-batch:{batch_size}',
             f'optim: 8bit-adam', f'weight-decay:{weight_decay}', f'gradient_accumulation_steps:{grad_accum_steps}',
             f'lora-r:{lora_r}', f'lora-alpha:{lora_alpha}'
             ]
        )





    print('a model has been pushed to the hub!')
    print('spray and pray worked this time.. :)')


# Perform grid search over all combinations of hyperparameters
def grid_search():
    i = 1
    # Use itertools.product to generate all combinations
    for lr, batch_size, weight_decay, grad_steps, lora_r, lora_alpha, lora_dropout in itertools.product(
        learning_rates, batch_sizes, weight_decays, grad_accumulation_steps, lora_ranks, lora_alphas, lora_dropouts
    ):
        print(f'model number: {i}')
        print(f"Training with lr={lr}, batch_size={batch_size}, weight_decay={weight_decay}, grad_steps={grad_steps}, lora_r={lora_r}, lora_alpha={lora_alpha}, lora_dropout={lora_dropout}")
        train_model(lr, batch_size, weight_decay, grad_steps, lora_r, lora_alpha, lora_dropout, check_point='aubmindlab/aragpt2-large', dataset_path='ahmadAlrabghi/al_tadmoreyyah')
        i+=1


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.9 µs


In [16]:
%%time
# run the grid search
# Warning: This grid search is going to take a long time to run! :)
grid_search()

model number: 1
Training with lr=5e-05, batch_size=8, weight_decay=0.01, grad_steps=2, lora_r=32, lora_alpha=64, lora_dropout=0.0
start trianing function


`low_cpu_mem_usage` was None, now set to True since model is quantized.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


training_started...


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
5,3.652200
10,3.595900
15,3.591400
20,3.605300
25,3.351200
30,3.550500
35,3.289200
40,3.507800
45,3.268400
50,3.349500


...

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/189M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

a model has been pushed to the hub!
spray and pray worked this time.. :)
CPU times: user 13h 18min, sys: 23min 11s, total: 13h 41min 11s
Wall time: 14h 5min 48s


In [ ]:
# # check and compare models performance in this notebook if you want
# %load_ext tensorboard
# %tensorboar --logdir logs

In [ ]:
%%time
# zipping the logs files so we can easily save it
! zip logs.zip logs
! cp logs.zip /content/drive/MyDrive/altadmoreyyah_model/

from google.colab import files
files.download('logs.zip')